# Experiment 1: Network Size Experiments
This notebook contains templates for network size experiments with different configurations:
- Tasks: Regression, Binary Classification, and Image Classification
- Network Types: Centralised and Decentralised
- Network Sizes: 5, 10, 50, 100
- Dirichlet alpha: 1 (fixed)

In [1]:
import sys
sys.path.append("../..")
import torch
import torch.nn as nn
from torch.optim import SGD
import asyncio
import nest_asyncio
from flwr_datasets.partitioner import DirichletPartitioner
from moxi import create_experiment

# Initialize asyncio for Jupyter
nest_asyncio.apply()

/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Model Definitions
Define the models for each task type

In [2]:
# Regression Model
class RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.act_func = nn.ReLU()
        self.layer1 = nn.Linear(3, 24)  # 3 input features
        self.layer2 = nn.Linear(24, 1)  # 1 output
        
    def forward(self, x):
        x = self.act_func(self.layer1(x))
        x = self.layer2(x)
        return x

# Binary Classification Model
class BinaryClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(30, 64)
        self.act_func = nn.ReLU()
        self.layer2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.act_func(self.layer1(x))
        x = self.layer2(x)
        return x

# Image Classification Model
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        x = self.activation(self.maxpool(self.conv1(x)))
        x = self.activation(self.maxpool(self.conv2(x)))
        x = self.activation(self.maxpool(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

## Experiment Runner Function

In [3]:
async def execute_experiment(flnw, num_rounds:int, epochs_per_round:int, experiment_name:str):
    """
    Execute a federated learning experiment.
    
    Args:
        flnw: Federated learning network object
        num_rounds: Number of federated learning rounds
        epochs_per_round: Number of local epochs per round
        experiment_name: Name for the experiment
    """
    await flnw.train(num_rounds=num_rounds, epochs_per_round=epochs_per_round, experiment_name=experiment_name)
    print(f"Experiment '{experiment_name}' completed.")

## Experiment Configuration

In [4]:
# Experiment parameters
network_sizes = [5, 10, 50, 100]  # Network sizes to test
network_types = ["centralised", "decentralised"]  # Network architectures
task_types = [
    {"name": "regression", "model": RegressionModel, "criterion": nn.MSELoss},
    {"name": "binary_classification", "model": BinaryClassifierModel, "criterion": nn.BCEWithLogitsLoss},
    {"name": "image_classification", "model": CNNModel, "criterion": nn.CrossEntropyLoss}
]

## Run Experiments
Choose a specific configuration to run by uncommenting and modifying the code below.

In [ ]:
# Example: Centralised Regression with Network Size = 5
# Change these variables to run different experiments

task_type = task_types[0]  # 0: regression, 1: binary_classification, 2: image_classification
network_type = network_types[0]  # 0: centralised, 1: decentralised
size = network_sizes[0]  # Choose from network_sizes list

# Create the configuration
config = {
    "network_name": f"{network_type}_{task_type['name']}_size_{size}",
    "network_type": network_type,
    "model_type": "parametric",
    "ml_framework": "pytorch",
    "comms": "async",
    "federated_rounds": 5,
    "network_size": size,  # Variable parameter
    "metrics": ["mean_perfomance", "convergence"],
    "logger": "mlflow",
    "node_base_config": {
        "model": None,
        "learning_rate": 1e-3,
        "optimizer": SGD,
        "train_data": None,
        "val_data": None,
        "criterion": task_type["criterion"],
        "random_sampling": False,
        "n_epochs": 5,
        "batch_size": 16
    },
    "experiment_config": {
        "task": task_type["name"],
        "partitioner": DirichletPartitioner,
        "alpha": 1,  # Fixed alpha for Experiment 1
        "num_worker": 2 if task_type["name"] != "regression" else 1,
        "max_samples": 1000
    },
    "adjcency_matrix": None,
    "model": None
}

# Create the model
model = task_type["model"]()

# Create the experiment
print(f"Running experiment: {network_type} {task_type['name']} with network size {size}")
flnw = create_experiment(config, model)

# Run the experiment
await execute_experiment(
    flnw, 
    num_rounds=5,  # Set to desired number (20 for full experiment)
    epochs_per_round=1, 
    experiment_name=f"exp1_{network_type}_{task_type['name']}_size_{size}"
)

In [ ]:
# Custom experiment runner - select specific combinations
# Uncomment and modify to run specific experiment combinations

async def run_selected_experiments(selected_task_types=None, selected_network_types=None, selected_sizes=None):
    """
    Run experiments with specific parameter combinations
    
    Args:
        selected_task_types: List of task indices to run (e.g., [0, 1] for regression and binary classification)
        selected_network_types: List of network type indices to run (e.g., [0] for centralised only)
        selected_sizes: List of network sizes to run (e.g., [5, 10])
    """
    if selected_task_types is None:
        selected_task_types = range(len(task_types))
    if selected_network_types is None:
        selected_network_types = range(len(network_types))
    if selected_sizes is None:
        selected_sizes = network_sizes
        
    experiment_count = 0
    total_experiments = len(selected_task_types) * len(selected_network_types) * len(selected_sizes)
    start_time = time.time()
    
    print(f"Starting selected experiments: {total_experiments} total experiments to run")
    
    # Loop through selected task types
    for task_idx in selected_task_types:
        task_type = task_types[task_idx]
        
        # Loop through selected network types
        for net_idx in selected_network_types:
            network_type = network_types[net_idx]
            
            print(f"\n{'='*80}")
            print(f"Running experiments for {network_type} {task_type['name']}")
            print(f"{'='*80}\n")
            
            # Loop through selected network sizes
            for size in selected_sizes:
                experiment_count += 1
                
                print(f"\nExperiment {experiment_count}/{total_experiments}")
                print(f"Task: {task_type['name']}, Network: {network_type}, Size: {size}")
                
                # Create configuration
                config = {
                    "network_name": f"{network_type}_{task_type['name']}_size_{size}",
                    "network_type": network_type,
                    "model_type": "parametric",
                    "ml_framework": "pytorch",
                    "comms": "async",
                    "federated_rounds": 5,
                    "network_size": size,
                    "metrics": ["mean_perfomance", "convergence"],
                    "logger": "mlflow",
                    "node_base_config": {
                        "model": None,
                        "learning_rate": 1e-3,
                        "optimizer": SGD,
                        "train_data": None,
                        "val_data": None,
                        "criterion": task_type["criterion"],
                        "random_sampling": False,
                        "n_epochs": 5,
                        "batch_size": 16
                    },
                    "experiment_config": {
                        "task": task_type["name"],
                        "partitioner": DirichletPartitioner,
                        "alpha": 1,
                        "num_worker": 2 if task_type["name"] != "regression" else 1,
                        "max_samples": 1000
                    },
                    "adjcency_matrix": None,
                    "model": None
                }
                
                try:
                    # Create model
                    model = task_type["model"]()
                    
                    # Create experiment
                    print(f"Creating experiment: {network_type} {task_type['name']} with network size {size}")
                    flnw = create_experiment(config, model)
                    
                    # Execute experiment
                    print(f"Running experiment...")
                    await execute_experiment(
                        flnw, 
                        num_rounds=5,  # Set to 20 for full experiment, 5 for testing
                        epochs_per_round=1,
                        experiment_name=f"exp1_{network_type}_{task_type['name']}_size_{size}"
                    )
                    
                except Exception as e:
                    print(f"Error in experiment {experiment_count}: {e}")
                
                # Wait briefly between experiments
                await asyncio.sleep(2)
    
    total_time = time.time() - start_time
    print(f"\n{'='*80}")
    print(f"Selected experiments completed!")
    print(f"Total time: {total_time:.1f} seconds")
    print(f"{'='*80}\n")

# Example usage:
# Run only regression task with both network types but only sizes 5 and 10
# await run_selected_experiments(
#     selected_task_types=[0],  # 0: regression
#     selected_network_types=[0, 1],  # Both centralised and decentralised
#     selected_sizes=[5, 10]  # Only smaller network sizes
# )

# Run all tasks but only for centralised networks with size 5
# await run_selected_experiments(
#     selected_network_types=[0],  # 0: centralised
#     selected_sizes=[5]  # Only size 5
# )

## Automated Experiment Loop
This cell runs all experiments for a specific network type and task type.
Uncomment and modify to run a batch of experiments.

In [ ]:
# Run all experiments for all task types and network types
import asyncio
import time
import datetime

async def run_all_experiments():
    """Run all network size experiments for all task types and network architectures"""
    
    experiment_count = 0
    total_experiments = len(task_types) * len(network_types) * len(network_sizes)
    start_time = time.time()
    
    print(f"Starting all experiments: {total_experiments} total experiments to run")
    print(f"Start time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # for all task types
    for task_idx, task_type in enumerate(task_types):
        # for network types
        for net_idx, network_type in enumerate(network_types):
            print(f"\n{'='*80}")
            print(f"Running experiments for {network_type} {task_type['name']}")
            print(f"{'='*80}\n")
            
            # for all network sizes
            for size_idx, size in enumerate(network_sizes):
                experiment_count += 1
                
                print(f"\nExperiment {experiment_count}/{total_experiments}")
                print(f"Task: {task_type['name']}, Network: {network_type}, Size: {size}")
                
                # Create configuration
                config = {
                    "network_name": f"{network_type}_{task_type['name']}_size_{size}",
                    "network_type": network_type,
                    "model_type": "parametric",
                    "ml_framework": "pytorch",
                    "comms": "async",
                    "federated_rounds": 5,
                    "network_size": size,
                    "metrics": ["mean_perfomance", "convergence"],
                    "logger": "mlflow",
                    "node_base_config": {
                        "model": None,
                        "learning_rate": 1e-3,
                        "optimizer": SGD,
                        "train_data": None,
                        "val_data": None,
                        "criterion": task_type["criterion"],
                        "random_sampling": False,
                        "n_epochs": 5,
                        "batch_size": 16
                    },
                    "experiment_config": {
                        "task": task_type["name"],
                        "partitioner": DirichletPartitioner,
                        "alpha": 1,
                        "num_worker": 2 if task_type["name"] != "regression" else 1,
                        "max_samples": 1000
                    },
                    "adjcency_matrix": None,
                    "model": None
                }
                
                try:
                    # intit model
                    model = task_type["model"]()
                    
                    # Create experiment
                    print(f"Creating experiment: {network_type} {task_type['name']} with network size {size}")
                    flnw = create_experiment(config, model)
                    
                    # run
                    print(f"Running experiment...")
                    await execute_experiment(
                        flnw, 
                        num_rounds=5,  # Set to 20 for full experiment, 5 for testing
                        epochs_per_round=1,
                        experiment_name=f"exp1_{network_type}_{task_type['name']}_size_{size}"
                    )
                    
                    # Calculate elapsed time
                    elapsed = time.time() - start_time
                    avg_time_per_exp = elapsed / experiment_count
                    remaining = avg_time_per_exp * (total_experiments - experiment_count)
                    
                    # Print progress
                    print(f"\nCompleted experiment {experiment_count}/{total_experiments}")
                    print(f"Elapsed time: {elapsed:.1f} seconds")
                    print(f"Estimated time remaining: {remaining:.1f} seconds")
                    print(f"Estimated completion time: {datetime.datetime.now() + datetime.timedelta(seconds=remaining)}")
                    
                except Exception as e:
                    print(f"Error in experiment {experiment_count}: {e}")
                
                # Wait briefly between experiments
                await asyncio.sleep(2)
    
    # Print completion summary
    total_time = time.time() - start_time
    print(f"\n{'='*80}")
    print(f"All experiments completed!")
    print(f"Total time: {total_time:.1f} seconds")
    print(f"End time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*80}\n")



In [6]:
await run_all_experiments()

Starting all experiments: 24 total experiments to run
Start time: 2025-09-09 06:57:15

Running experiments for centralised regression


Experiment 1/24
Task: regression, Network: centralised, Size: 5
Creating experiment: centralised regression with network size 5


/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: SIH/palmer-penguins.
  warnings.warn(
2025/09/09 06:57:18 INFO mlflow.tracking.fluent: Experiment with name 'exp1_centralised_regression_size_5' does not exist. Creating a new experiment.
2025/09/09 06:57:18 INFO mlflow.tracking.fluent: Experiment with name 'exp

Running experiment...
Training Complete!
Experiment 'exp1_centralised_regression_size_5' completed.

Completed experiment 1/24
Elapsed time: 11.8 seconds
Estimated time remaining: 271.5 seconds
Estimated completion time: 2025-09-09 07:01:59.293297
Training Complete!
Experiment 'exp1_centralised_regression_size_5' completed.

Completed experiment 1/24
Elapsed time: 11.8 seconds
Estimated time remaining: 271.5 seconds
Estimated completion time: 2025-09-09 07:01:59.293297

Experiment 2/24
Task: regression, Network: centralised, Size: 10
Creating experiment: centralised regression with network size 10

Experiment 2/24
Task: regression, Network: centralised, Size: 10
Creating experiment: centralised regression with network size 10


/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: SIH/palmer-penguins.
  warnings.warn(
2025/09/09 06:57:31 INFO mlflow.tracking.fluent: Experiment with name 'exp1_centralised_regression_size_10' does not exist. Creating a new experiment.
2025/09/09 06:57:31 INFO mlflow.tracking.fluent: Experiment with name 'ex

Running experiment...
Training Complete!
Experiment 'exp1_centralised_regression_size_10' completed.

Completed experiment 2/24
Elapsed time: 26.3 seconds
Estimated time remaining: 289.5 seconds
Estimated completion time: 2025-09-09 07:02:31.764994
Training Complete!
Experiment 'exp1_centralised_regression_size_10' completed.

Completed experiment 2/24
Elapsed time: 26.3 seconds
Estimated time remaining: 289.5 seconds
Estimated completion time: 2025-09-09 07:02:31.764994

Experiment 3/24
Task: regression, Network: centralised, Size: 50
Creating experiment: centralised regression with network size 50

Experiment 3/24
Task: regression, Network: centralised, Size: 50
Creating experiment: centralised regression with network size 50


...

Running experiment...
Training Complete!
Experiment 'exp1_decentralised_image_classification_size_10' completed.

Completed experiment 22/24
Elapsed time: 1579.9 seconds
Estimated time remaining: 143.6 seconds
Estimated completion time: 2025-09-09 07:25:59.438717
Training Complete!
Experiment 'exp1_decentralised_image_classification_size_10' completed.

Completed experiment 22/24
Elapsed time: 1579.9 seconds
Estimated time remaining: 143.6 seconds
Estimated completion time: 2025-09-09 07:25:59.438717

Experiment 23/24
Task: image_classification, Network: decentralised, Size: 50
Creating experiment: decentralised image_classification with network size 50
Loading federated CIFAR-10 dataset: uoft-cs/cifar10

Experiment 23/24
Task: image_classification, Network: decentralised, Size: 50
Creating experiment: decentralised image_classification with network size 50
Loading federated CIFAR-10 dataset: uoft-cs/cifar10


2025/09/09 07:23:39 INFO mlflow.tracking.fluent: Experiment with name 'exp1_decentralised_image_classification_size_50' does not exist. Creating a new experiment.


Running experiment...
Training Complete!
Experiment 'exp1_decentralised_image_classification_size_50' completed.

Completed experiment 23/24
Elapsed time: 2051.9 seconds
Estimated time remaining: 89.2 seconds
Estimated completion time: 2025-09-09 07:32:57.045542
Training Complete!
Experiment 'exp1_decentralised_image_classification_size_50' completed.

Completed experiment 23/24
Elapsed time: 2051.9 seconds
Estimated time remaining: 89.2 seconds
Estimated completion time: 2025-09-09 07:32:57.045542

Experiment 24/24
Task: image_classification, Network: decentralised, Size: 100
Creating experiment: decentralised image_classification with network size 100
Loading federated CIFAR-10 dataset: uoft-cs/cifar10

Experiment 24/24
Task: image_classification, Network: decentralised, Size: 100
Creating experiment: decentralised image_classification with network size 100
Loading federated CIFAR-10 dataset: uoft-cs/cifar10


2025/09/09 07:31:32 INFO mlflow.tracking.fluent: Experiment with name 'exp1_decentralised_image_classification_size_100' does not exist. Creating a new experiment.


Running experiment...
Training Complete!
Experiment 'exp1_decentralised_image_classification_size_100' completed.

Completed experiment 24/24
Elapsed time: 2714.1 seconds
Estimated time remaining: 0.0 seconds
Estimated completion time: 2025-09-09 07:42:30.035247
Training Complete!
Experiment 'exp1_decentralised_image_classification_size_100' completed.

Completed experiment 24/24
Elapsed time: 2714.1 seconds
Estimated time remaining: 0.0 seconds
Estimated completion time: 2025-09-09 07:42:30.035247

All experiments completed!
Total time: 2716.1 seconds
End time: 2025-09-09 07:42:32


All experiments completed!
Total time: 2716.1 seconds
End time: 2025-09-09 07:42:32

